In [36]:
import yfinance as yf
from data.fetch_data import fetch_pdf_info
import pandas as pd

my_etf_tkr = "AIQ"

pdf_info = fetch_pdf_info(etf_tkr=my_etf_tkr)

pdf_df = pd.DataFrame(pdf_info)

tkr_list = pdf_df.sort_values(by="float_shares", ascending=False).head().reset_index(drop=True)['child_stk_tkr'].to_list()

stk_infos = yf.Tickers(tkr_list)



['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'ERIC']


In [30]:
from tqdm.auto import tqdm

dict_list = []

for tkr in tqdm(tkr_list):
    
    a_tkr_dict = {}
    
    a_tkr_dict['stock'] = tkr

    a_tkr_dict['news_titles'] = []

    for news in stk_infos.tickers[tkr].news:
        
        a_tkr_dict['news_titles'].append(news['title'])

    dict_list.append(a_tkr_dict)

100%|██████████| 5/5 [00:02<00:00,  1.76it/s]


In [31]:
str(dict_list)

'[{\'stock\': \'AAPL\', \'news_titles\': [\'15 Most Bike Friendly Cities in the US\', \'Hollywood strikes, subscriber churn, sports rights: Big media woes\', \'Apple’s Impact Accelerator picks 2 Bay Area sustainability startups\', \'Why 3D Systems Stock Zoomed Higher Today\', \'5 Monopoly Stocks At the Top of Their Game\', \'Top Research Reports for Apple, Broadcom & Caterpillar\', \'Economy: We have a ‘goldilocks scenario,’ strategist says\', \'Apple Stock on Pace for Worst Month of the Year\']}, {\'stock\': \'GOOGL\', \'news_titles\': ["Privacy activist Schrems files complaints against Google\'s Fitbit", \'Google introduces generative AI to Search in India, Japan\', "Google\'s AI-powered search expands outside U.S. to India and Japan", \'Google is bringing its generative AI search engine to India, Japan\', "Factbox-Big names in Big Tech to attend AI forum of US Senate\'s Schumer", \'Nvidia Partner Expects AI Server Sales to Double in 2024\', \'Don’t want to go to a work meeting? Goog

In [32]:
import os, json

with open('conf.json', 'r') as f:
    json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['API_KEY']

In [33]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                # "You are now acting as an intelligent fund manager who is fluent in both English and Korean. I will provide you with news headlines for 8 different stock tickers. Your task is to identify the common themes or keywords that appear across all these different tickers, not just within each individual one. Please summarize these overarching themes in Korean, in a coherent format limited to 10 sentences. Ensure your response is tailored to be informative for stock investors and is presented in a professional manner."
                "You are now acting as an intelligent fund manager fluent in both English and Korean. I will provide you with news headlines of different stock tickers. Your task is to identify common themes that appear across all these different tickers, not just within each individual one. Please summarize these overarching themes in Korean, in a coherent format limited to 10 sentences. Ensure your response is tailored to be informative for stock investors and is presented in a professional manner. Additionally, please conclude your answer with a noun. don't answer in bullet points"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm(template.format_messages(text=str(dict_list)))

AIMessage(content='주식 시장의 주요 테마를 파악하기 위해 AAPL, GOOGL, AMZN, MSFT 및 ERIC의 뉴스 헤드라인을 조사한 결과, 다음과 같은 주요 테마를 도출할 수 있습니다:\n\n1. 인공지능 (AI) 및 기계 학습 기술의 적용: AAPL, GOOGL, MSFT 등의 기업은 AI를 활용한 검색 엔진 및 기술 개발에 집중하고 있습니다.\n\n2. 지속 가능성 및 환경 문제: AAPL은 Impact Accelerator를 통해 환경 지속 가능성에 기여하는 스타트업을 지원하고 있습니다.\n\n3. 클라우드 컴퓨팅 및 플랫폼: MSFT와 ERIC은 AWS와의 협업 및 Google Cloud와의 기술 파트너로서 클라우드 인프라 및 솔루션을 개발하고 있습니다.\n\n4. 전자 상거래 및 온라인 플랫폼: AMZN과 Shopify는 판매자에게 Amazon Logistics 및 Prime 옵션을 제공하여 전자 상거래 영역에서 협력하고 있습니다.\n\n5. 글로벌 라이선스 및 특허: ERIC은 Huawei와 글로벌 특허 라이선스 계약을 갱신하며 기술적 협력을 이어가고 있습니다.\n\n6. 주식 시장 동향: AAPL은 올해 최악의 월을 기록하는 등 주식 시장에서 변동성이 큰 모습을 보이고 있습니다.\n\n7. AI 기술의 확장: GOOGL은 AI를 검색 엔진에 적용하고 인도와 일본에서도 서비스를 확장하고 있습니다.\n\n8. 통신 기술: ERIC은 통신 업계에서 기술 파트너로서 선정되고, 최근에는 통신 업계의 혁신을 위한 기술적 역량을 인정받았습니다.\n\n9. 경제 상황: "골디락스 시나리오"라는 용어를 사용하여 경제적 안정성을 강조하는 기사들이 다수 보도되었습니다.\n\n10. 기술 기업의 주가 움직임: AAPL, GOOGL, MSFT 및 ERIC은 주식 시장에서 주가 상승폭을 보이며 시장 성과를 앞섰습니다.\n\n이러한 테마들은 주식 투자자들이 기업의 비즈니스 전략과 주식 시장 동향을 파악하는 데 도움을 줄 수 있습니다.', addition